<a href="https://colab.research.google.com/github/LunaticRuri/cpng-reviews-nlp/blob/master/doc2vec_%EA%B8%B0%EB%B0%98_%EC%83%81%ED%92%88_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전 설치

In [ ]:
print("install package")

install package


In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip install JPype1
pip install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
!pip install numpy --upgrade

In [ ]:
!pip install gensim --upgrade

# 구현

In [ ]:
from konlpy.tag import Mecab
import re


class Preprocessor:
    stop_pos = ['JX', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'EF',
                'JC', 'SY', 'EP', 'MM', 'NNBC', 'NP', 'EC', 'VCP', 'NMB', 'MAG']

    POS = "pos"
    MORPHS = "morphs"

    def __init__(self, mode):
        self.mecab = Mecab()

        if mode == Preprocessor.POS:
            self.mode = Preprocessor.POS
        else:
            self.mode = Preprocessor.MORPHS

    @staticmethod
    def pos_remover(token_pos_tuple):
        """
        if token_pos_tuple[1] == 'MAG' and token_pos_tuple[0] == '안':
            return False
        """

        if token_pos_tuple[1] in Preprocessor.stop_pos:
            return True
        else:
            return False

    @staticmethod
    def only_hangul_space(raw_text):
        r_hangul = re.sub(r'[^ㄱ-ㅣ가-힣ㅣ\s\d]', "", raw_text)
        return r_hangul

    @staticmethod
    def pos_to_morphs(pos_list):
        output_list = [elem[0] for elem in pos_list]
        return output_list

    def preprocessing(self, raw_text):
        hangul = self.only_hangul_space(raw_text)
        pos_list = self.mecab.pos(hangul)
        output_list = [elem for elem in pos_list if not self.pos_remover(elem)]

        if self.mode == Preprocessor.MORPHS:
            output_list = self.pos_to_morphs(output_list)

        return output_list


In [ ]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import json
import pickle
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedLineDocument
import multiprocessing


def hangul_preprocessing(raw_text):
    r_hangul = re.sub(r'[^ㄱ-ㅣ가-힣ㅣ\s+]', "", raw_text)
    return r_hangul


def json_to_list(json_elem):
    sum_of_reviews: str = ""

    for pair in json_elem["reviews"]:
        data = pair["data"]
        if not data:
            continue
        pp = hangul_preprocessing(data)
        if len(pp) >= 8000:
            pp = pp[:8000]
        sum_of_reviews += pp

    if len(sum_of_reviews) >= 10000:
        output_list = [json_elem['product_id'], json_elem['product_name'], sum_of_reviews]
    else:
        output_list = []

    return output_list


def new_model():
    reviews_path = "./drive/MyDrive/data/0abcdef.json"

    with open(reviews_path, 'r') as f:
        product_reviews = json.load(f)

    # id, name, reviews

    th_list = []

    for elem in tqdm(product_reviews.values(), total=len(product_reviews)):
        result = json_to_list(elem)
        if result:
            th_list.append(result)

    print("\n th_list len", len(th_list))

    pp = Preprocessor(mode=Preprocessor.MORPHS)

    tagged_corpus_list = []

    tags_order = []

    for elem in tqdm(th_list, total=len(th_list)):
        tag_name_pair = (elem[0], elem[1])
        text = ' '.join(pp.preprocessing(elem[2]))

        tags_order.append(tag_name_pair)
        tagged_corpus_list.append(text)

    print(tags_order)

    with open("./data/cpng_0abcdef_tagged_line_document.space", 'w+') as fp:
        for item in tagged_corpus_list:
            fp.write(item + '\n')

    print("***********************************")

    tagged_docs = TaggedLineDocument("./data/cpng_0abcdef_tagged_line_document.space")

    model = doc2vec.Doc2Vec(
        vector_size=300,
        alpha=0.025,
        min_alpha=0.025,
        workers=multiprocessing.cpu_count(),
        window=5,
        min_count=5,
    )

    print("model.build_vocab")

    model.build_vocab(tagged_docs)

    print("model.train()")

    model.train(tagged_docs, total_examples=model.corpus_count, epochs=20)

    print("Saving data...")

    with open("./data/cpng_0abcdef_doc2vec_model.pickle", 'wb') as fp:
        pickle.dump(model, fp)
    with open("./data/cpng_0abcdef_doc2vec_model_tags_order.pickle", 'wb') as fp:
        pickle.dump(tags_order, fp)

    return model, tags_order


def open_model_tags_order():
    with open("./data/cpng_0abcdef_doc2vec_model.pickle", 'rb') as fp:
        model = pickle.load(fp)

    with open("./data/cpng_0abcdef_doc2vec_model_tags_order.pickle", 'rb') as fp:
        tags_order = pickle.load(fp)

    return model, tags_order


def run(model, tags_order):
    index = 10000
    similar_product = model.dv.most_similar(index)
    print(tags_order[index])
    print("****************************************")
    for elem in similar_product:
        print(tags_order[elem[0]], elem[1])

new_model()
model, tags_order = open_model_tags_order()
run(model, tags_order)

100%|██████████| 241521/241521 [00:20<00:00, 11558.94it/s]



 th_list len 31885


100%|██████████| 31885/31885 [25:22<00:00, 20.94it/s]


[(4577635015, '필립스 안전칼날 헤드 방수 코털제거기 NT3650/16'), (200199075, 'K1 방문 손잡이 심플 그레이'), (216143177, '모구모구 복숭아맛'), (335514912, '피직스가드 라이프 발목보호대 스킨 S M'), (16782052, '바디아 스모크드 파프리카 파우더'), (5761347660, '백제 멸치맛 쌀국수92g'), (108948008, '광복농산 하늘정원 찰현미'), (113285103, '즐거운가구 아이린1800렌지대 주방수납장 전자렌지대 렌지다이 렌지대 주방 수납장'), (309542223, '매드킨 [PN-072] 살튀방지 스왑 실리콘바벨 3mm 무알러지 무염증 피어싱'), (5293806397, '서래야 GAP 인증 블루베리'), (6340078488, '더바디샵 대용량 핑크 그레이프후룻 샤워 젤 과일향'), (2201895429, '대한 닌텐도 스위치 칩 카트리지 케이스 게임팩 보관함'), (250047092, '부케가르니 바디워시 베이비파우더'), (243957199, '맘스네이처 대나무 아기목욕타올 85 x 85 cm'), (2163978, '태화 고무장갑 S'), (3975404, '병도매닷컴 DN350 디스펜서용기 3개'), (19316611, '팬물산 그리드 등받이 폴딩스툴'), (2160757046, '네파 여성용 하이킹 단목양말 5켤레'), (6381976268, '이지쿠커 미니 와플팬'), (34824305, '스칸디앤홈 스윗볼 러그 카페트'), (6310373707, '하림 주원산오리 건강한오리 훈제 & 소스'), (1369651167, '구월의아침 국산100%쪄서볶은 서리태콩물가루 검은콩가루 500g 미숫가루 선식'), (5221704884, '황금열매 주스용 맛있는 완숙 일반 찰 토마토 쥬스용 5KG 10KG 소과 로얄과 중과 대과'), (4339951291, '보르미올리 보테 오로라 와인잔'), (1362292680, '쿠첸 원터치 클린커버 10인용 IH압력패킹'), (

별점 별로도 해보자? 4,5

In [ ]:
def json_to_list45(json_elem):
    sum_of_reviews: str = ""

    rating = 0
    for pair in json_elem["reviews"]:
        rating += 1
        if rating > 3:
            continue

        data = pair["data"]
        if not data:
            continue
        pp = hangul_preprocessing(data)
        if len(pp) >= 2000:
            pp = pp[:2000]
        sum_of_reviews += pp

    if len(sum_of_reviews) >= 2000:
        output_list = [json_elem['product_id'], json_elem['product_name'], sum_of_reviews]
    else:
        output_list = []

    return output_list

def new_model_45():
    reviews_path = "./drive/MyDrive/data/0abcdef.json"

    with open(reviews_path, 'r') as f:
        product_reviews = json.load(f)

    # id, name, reviews

    th_list = []

    for elem in tqdm(product_reviews.values(), total=len(product_reviews)):
        result = json_to_list45(elem)
        if result:
            th_list.append(result)

    print("\n th_list len", len(th_list))

    pp = Preprocessor(mode=Preprocessor.MORPHS)

    tagged_corpus_list = []

    tags_order = []

    for elem in tqdm(th_list, total=len(th_list)):
        tag_name_pair = (elem[0], elem[1])
        text = ' '.join(pp.preprocessing(elem[2]))

        tags_order.append(tag_name_pair)
        tagged_corpus_list.append(text)

    print(tags_order)

    with open("./data/cpng_0abcdef_123_tagged_line_document.space", 'w+') as fp:
        for item in tagged_corpus_list:
            fp.write(item + '\n')

    print("***********************************")

    tagged_docs = TaggedLineDocument("./data/cpng_0abcdef_123_tagged_line_document.space")

    model = doc2vec.Doc2Vec(
        vector_size=300,
        alpha=0.025,
        min_alpha=0.025,
        workers=multiprocessing.cpu_count(),
        window=5,
        min_count=5,
    )

    print("model.build_vocab")

    model.build_vocab(tagged_docs)

    print("model.train()")

    model.train(tagged_docs, total_examples=model.corpus_count, epochs=20)

    print("Saving data...")

    with open("./data/cpng_0abcdef_123_doc2vec_model.pickle", 'wb') as fp:
        pickle.dump(model, fp)
    with open("./data/cpng_0abcdef_123_doc2vec_model_tags_order.pickle", 'wb') as fp:
        pickle.dump(tags_order, fp)

    return model, tags_order


def open_model_tags_order_45():
    with open("./data/cpng_0abcdef_123_doc2vec_model.pickle", 'rb') as fp:
        model = pickle.load(fp)

    with open("./data/cpng_0abcdef_123_doc2vec_model_tags_order.pickle", 'rb') as fp:
        tags_order = pickle.load(fp)

    return model, tags_order


def run_45(model, tags_order):
    index = 10000
    similar_product = model.dv.most_similar(index)
    print(tags_order[index])
    print("****************************************")
    for elem in similar_product:
        print(tags_order[elem[0]], elem[1])

new_model_45()
model, tags_order = open_model_tags_order_45()
run_45(model, tags_order)

100%|██████████| 241521/241521 [00:05<00:00, 47513.62it/s]



 th_list len 15537


100%|██████████| 15537/15537 [04:03<00:00, 63.89it/s]


[(4577635015, '필립스 안전칼날 헤드 방수 코털제거기 NT3650/16'), (200199075, 'K1 방문 손잡이 심플 그레이'), (113285103, '즐거운가구 아이린1800렌지대 주방수납장 전자렌지대 렌지다이 렌지대 주방 수납장'), (309542223, '매드킨 [PN-072] 살튀방지 스왑 실리콘바벨 3mm 무알러지 무염증 피어싱'), (5293806397, '서래야 GAP 인증 블루베리'), (250047092, '부케가르니 바디워시 베이비파우더'), (2163978, '태화 고무장갑 S'), (3975404, '병도매닷컴 DN350 디스펜서용기 3개'), (6381976268, '이지쿠커 미니 와플팬'), (34824305, '스칸디앤홈 스윗볼 러그 카페트'), (6333456681, '엔비사과 주스용 10kg 쥬스용 사과 못난이 가정용 흠집 흠과 잼용 쨈'), (216555858, '너트리 건자두'), (35310598, '엘림 면장갑40g'), (11660555, '바디아 갈릭 파우더'), (28755540, '보니타 대형빈백 파우치 스웨이드 P202'), (2437845, '농심 사리곰탕면 봉지 110g'), (180193500, '[텐바이텐] 이비자 화이트 물티슈 케이스'), (27917, '나우푸드 칼슘 구연산염 타블렛 글루텐 프리'), (171671, '코시나 바람폭풍 점보 샤워기'), (5729364191, '헤트라스 프리미엄 대용량 디퓨저 500ml 3개 + 선물세트'), (6082231413, '케이안 슬림 마스터 강화유리 휴대폰 액정보호필름 + 가이드 툴 세트'), (171445596, '쿤달 헤어바디 4구 명절 선물세트 C 베이비파우더향'), (5376121095, '맥 매트 립스틱'), (6441698050, '[공식수입] 랑방 에끌라 드 아르페쥬 우먼 오 드 퍼퓸'), (116684794, '풀무원 참깨 흑임자 드레싱'), (334186496, '유리아쥬 진피 데일리 페미닌 마일드 젤 여성청결제'), (258